In [1]:
!pip install string2string

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00

In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
import re
drive.mount('/content/drive')
workdir = '/content/drive/MyDrive/Sumerian Network' # for Tina

Mounted at /content/drive


### BERTScore and BARTScore for Semantic Similarity

In [3]:
# Import the BERTScore and BARTScore classes from the misc module
from string2string.similarity import BERTScore, BARTScore

# Create an instance of the BERTScore class
bert_score = BERTScore(
    model_name_or_path='bert-base-uncased',
)

# Create an instance of the BARTScore class (if device is not specified, it will be automatically detected)
bart_score = BARTScore(
    model_name_or_path='facebook/bart-large-cnn',
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
etcsri_words_df = pd.read_csv("/content/drive/MyDrive/geography/etcsri_words_df.csv")
eng_txt = pd.read_csv("/content/drive/MyDrive/geography/etcsri_Q000377.txt",sep='delimiter', header=None)
eng_txt

<ipython-input-13-967996b229ee>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  eng_txt = pd.read_csv("/content/drive/MyDrive/geography/etcsri_Q000377.txt",sep='delimiter', header=None)


,0
0,(Cylinder A i 1) On the day when in heaven and...
1,"(Cylinder A i 9) ""Its owner requested the temp..."
2,"(Cylinder A i 17) And on this day, in a noctur..."
3,"(Cylinder A i 24) ""Well, I am going to tell he..."
4,"(Cylinder A ii 4) He stepped aboard his boat, ..."
...,...
99,(Cylinder B xvii 18) When his master entered t...
100,(Cylinder B xviii 4) He observed the laws of N...
101,(Cylinder B xxi 17) ... determined a fate for ...
102,(Cylinder B xxii 16) .... At your name cow-pen...


In [14]:
etcsri_words_df.columns

Index(['Unnamed: 0', 'extent', 'scope', 'state', 'id_word', 'lemma', 'id_text',
       'lang', 'form', 'gdl', 'cf', 'gw', 'sense', 'norm', 'pos', 'epos',
       'base', 'morph', 'morph2', 'label', 'delim', 'cont', 'syntax_ub-after',
       'id_line', 'norm1'],
      dtype='object')

In [7]:
gw = etcsri_words_df['gw']
gw

0          NaN
1            1
2        water
3         cvve
4            1
         ...  
29914        1
29915    build
29916        1
29917      NaN
29918      big
Name: gw, Length: 29919, dtype: object

In [16]:
# Import the NeedlemanWunsch class from the alignment module
from string2string.alignment import NeedlemanWunsch

# Create an instance of the NeedlemanWunsch class
nw = NeedlemanWunsch()

# Define two strings (e.g., DNA sequences)
seq1 = gw
seq2 = eng_txt[0][0]

# Compute the alignment between two strings
aligned_seq1, aligned_seq2, score_matrix = nw.get_alignment(seq1, seq2, return_score_matrix=True)

# Print the alignment
print(f'Here is the alignment between "{seq1}" and "{seq2}", as computed by the Needleman-Wunsch algorithm:')
nw.print_alignment(aligned_seq1, aligned_seq2)

TypeError: ignored